In [6]:
# OneR Algorithm
# For each predictor,
#      For each value of that predictor, make a rule as follows;
#            Count how often each value of target (class) appears
#            Find the most frequent class
#            Make the rule assign that class to this value of the predictor
#      Calculate the total error of the rules of each predictor
# Choose the predictor with the smallest total error.

import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

#Загрузка данных об ирисах
dataset = load_iris()
#Массив с примерами по 4 признака
X = dataset.data
#Массив с классами
y = dataset.target
#Количество примеров - 150, признаков - 4
n_samples, n_features = X.shape

#Математическое ожидаине для каждого из признаков
attribute_means = X.mean(axis=0) #axis=0 - считать по столбцам, axis=1 - считать по строкам
#проверка что количесвто признаков соответствует заданному
assert attribute_means.shape == (n_features,)
#переход от continious features к categorical features. Т.е. преобразование признаков с помощью пооговой функции
#в качестве порога - матожидание
#т.е. бинаризация признаков
X_d = np.array(X >= attribute_means, dtype='int')
random_state = 14  # так, чтобы получить одинак. рез-ты
#разделение на тестовую и обучающие выборки
X_train, X_test, y_train, y_test = train_test_split(X_d, y, random_state=random_state)
print("There are {} training samples".format(y_train.shape))
print("There are {} testing samples".format(y_test.shape))

from collections import defaultdict
from operator import itemgetter

def train(X, y_true, feature):
    # Check that variable is a valid number - проверка валидный ли номер признака
    n_samples, n_features = X.shape
    assert 0 <= feature < n_features
    # Get all of the unique values that this variable has - получение всех уникальных знаачений для признака 
    # после бинаризации значение 1 и 0
    values = set(X[:,feature])
    # Stores the predictors array that is returned
    predictors = dict()
    errors = []
    #для каждого из значений признака вычисляется:
    #most_frequent_class  класс, который чаще встречается при данном значении признака
    #error - ошибка при подоюном правиле
    for current_value in values:
        most_frequent_class, error = train_feature_value(X, y_true, feature, current_value)
        predictors[current_value] = most_frequent_class
        errors.append(error)
    # Compute the total error of using this feature to classify on
    total_error = sum(errors)
    return predictors, total_error

# Compute what our predictors say each sample is based on its value
#y_predicted = np.array([predictors[sample[feature]] for sample in X])
    

def train_feature_value(X, y_true, feature, value):
    # Create a simple dictionary to count how frequency they give certain predictions
    class_counts = defaultdict(int)
    # Iterate through each sample and count the frequency of each class/value pair
    # для каждого значения признака считаются появляющиеся классы
    for sample, y in zip(X, y_true):
        if sample[feature] == value:
            class_counts[y] += 1       
    # Now get the best one by sorting (highest first) and choosing the first item
    # выбор наиболее частовстречающегося класса - считаем, что данное значение данного признака 
    # соотеветствует этому классу
    sorted_class_counts = sorted(class_counts.items(), key=itemgetter(1), reverse=True)
    most_frequent_class = sorted_class_counts[0][0]
    # The error is the number of samples that do not classify as the most frequent class
    # *and* have the feature value.
    n_samples = X.shape[1] #выгляди как ненужная строка
    # ошибка - количесвто примеров, где класс другой
    error = sum([class_count for class_value, class_count in class_counts.items()
                 if class_value != most_frequent_class])
    return most_frequent_class, error

# Compute all of the predictors
# вычислили правило и ошибку для каждого признака
all_predictors = {variable: train(X_train, y_train, variable) for variable in range(X_train.shape[1])}
errors = {variable: error for variable, (mapping, error) in all_predictors.items()}
# Now choose the best and save that as "model"
# Sort by error
# выбрали наилучшее по минимальной ошибке
best_variable, best_error = sorted(errors.items(), key=itemgetter(1))[0]
print("The best model is based on variable {0} and has error {1:.2f}".format(best_variable, best_error))

# Choose the bset model
model = {'variable': best_variable,
         'predictor': all_predictors[best_variable][0]}
print(model)

There are (112,) training samples
There are (38,) testing samples
The best model is based on variable 2 and has error 37.00
{'variable': 2, 'predictor': {0: 0, 1: 2}}


In [7]:
#функция для получения предсказаний по выбранной модели
def predict(X_test, model):
    variable = model['variable']
    predictor = model['predictor']
    y_predicted = np.array([predictor[int(sample[variable])] for sample in X_test])
    return y_predicted
y_predicted = predict(X_test, model)
print(y_predicted)

[0 0 0 2 2 2 0 2 0 2 2 0 2 2 0 2 0 2 2 2 0 0 0 2 0 2 0 2 2 0 0 0 2 0 2 0 2
 2]


In [8]:
# Compute the accuracy by taking the mean of the amounts that y_predicted is equal to y_test
accuracy = np.mean(y_predicted == y_test) * 100
print("The test accuracy is {:.1f}%".format(accuracy))

The test accuracy is 65.8%


In [9]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predicted))
# для каждого из классов 0, 1, 2 - (для 1 - нули, потому что по получекнному классификатору
# это класс не получается в результате, потому же и выскакиевает предупреждение)
# True Positives (TP) True Negatives (TN) False Positives (FP) False Negatives (FN) 
# precision - отношение правильно положительно предсказанных ко всем правильно предсказанным (TP/TP+FP)
# recall  - отношение правильно положительно предсказанным ко всем, кто фактически должен быть положительным (TP/TP+FN)
# F1 score - F1 Score is the weighted average of Precision and Recall ( 2*(Recall * Precision) / (Recall + Precision))
# macro avg - среднее арефметическое хар-к (пример по precision ((0.94 + 0.00 + 0.40)/3)
# weighted avg - среднее арефметическое хар-к с учетем веса для каждоко класса (т.е для 0 - 17/38)

              precision    recall  f1-score   support

     class 0       0.94      1.00      0.97        17
     class 1       0.00      0.00      0.00        13
     class 2       0.40      1.00      0.57         8

    accuracy                           0.66        38
   macro avg       0.45      0.67      0.51        38
weighted avg       0.51      0.66      0.55        38



c:\users\orus-kade\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.5047368421052632